In [4]:
loadModel(Modelica) 

True

In [5]:
model cylinder "play"
    
  import Modelica.Constants.*;
  import Modelica.SIunits.{Angle,AngularVelocity,Length,Diameter,Volume,Pressure,Temperature};
  import Modelica.SIunits.Conversions.NonSIunits.{Angle_deg,AngularVelocity_rpm};
  import Modelica.SIunits.Conversions.{from_deg,from_rpm};
  import Modelica.Media;

     
  parameter AngularVelocity_rpm N_rpm=600 "engine speed [rpm]";
  parameter Diameter bore(displayUnit="mm")=0.080 "bore [m]";
  parameter Length stroke(displayUnit="mm")=0.090 "stroke [m]";
  parameter Length conrod(displayUnit="mm")=0.140 "connecting rod length [m]";
  parameter Pressure P_bdc(displayUnit="bar")=100000 "pressure at BDC [Pa]";
  parameter Temperature T_bdc=300 "temp at BDC [K]";
  parameter Real n=1.3 "polytropic exponent";
  parameter Real rc=12.5 "compression ratio";
  parameter Volume Vd(displayUnit="l") = pi/4*bore^2*stroke "displacement";  
  parameter Volume V_tdc(displayUnit="l") = Vd/(rc-1) "clearance (min) volume";
  parameter Volume V_bdc(displayUnit="l") = V_tdc*rc "max volume";  

// variables
  Angle_deg degCA(start=0) "crank angle [deg]. Zero degrees is TDC";
  Angle_deg degCAmod360 "crank angle [deg]";
  Angle CA "crank angle [rad]";
  Angle theta "conrod angle from cylinder axis [rad]";
  AngularVelocity N "engine speed [rad/s]";
  Volume V_cyl(displayUnit="l") "instantaneous cylinder volume";
  Pressure P_cyl(displayUnit="bar") "cylinder pressure [Pa]";
  Temperature T_cyl "cylinder temperature [K]";
  Length x(displayUnit="mm") "piston relative distance from TDC";
  package AIR = Modelica.Media.Air.DryAirNasa; 
//   CO2.BaseProperties medium; 
    
  
// initial equation
//  degCA = 90;
   
equation
// define crank angle; display it in degrees, calculate it always in radians 
  CA = from_deg(degCA);
  N = from_rpm(N_rpm);
  der(CA) = N;
  degCAmod360 = mod(degCA,360);
  //degCA=mod(degCA,720);
  
// slider-crank geometry
  sin(CA)/conrod = sin(theta)/(stroke/2) "Law of sines";
  x = (( conrod+stroke/2 ))
     -(( conrod*cos(theta)+cos(CA)*stroke/2 )) "x=0 @TDC; x=S @BDC";
  V_cyl = V_tdc + x*pi/4*bore^2;
 
// let's add some thermodynamics
  P_cyl/P_bdc = (V_bdc/V_cyl)^n  "polytropic relationship";
  P_cyl*V_cyl/T_cyl = P_bdc*V_bdc/T_bdc "ideal gas law";
  
  
end cylinder;


('cylinder',)

In [6]:
simulate(cylinder, stopTime = 0.35)

{'resultFile': '/user/jovyan/work/cylinder_res.mat', 'simulationOptions': "startTime = 0.0, stopTime = 0.35, numberOfIntervals = 500, tolerance = 1e-06, method = 'dassl', fileNamePrefix = 'cylinder', options = '', outputFormat = 'mat', variableFilter = '.*', cflags = '', simflags = ''", 'messages': 'LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n', 'timeFrontend': 0.3470016, 'timeBackend': 0.0075582, 'timeSimCode': 0.0103578, 'timeTemplates': 0.0346457, 'timeCompile': 1.0970359, 'timeSimulation': 0.05495379999999994, 'timeTotal': 1.5527102}

In [7]:
plot(T_cyl)